In [1]:
import requests

from portfoliobuilder import api_utils, finnhub_endpoint, finnhub_key

### Notes for AAPL 2021 10-K
Year: response.json()['data'][0]['year']

Form: response.json()['data'][0]['form']

Report: response.json()['data'][0]['report']

Balance Sheet: response.json()['data'][0]['report']['bs']

Income Statement: response.json()['data'][0]['report']['ic']

Cash Flow Statement: response.json()['data'][0]['report']['cf']

Total Revenue: response.json()['data'][0]['report']['ic'][0]['value']

In [3]:
def get_fins_as_reported(symbol):
    url = finnhub_endpoint + 'stock/financials-reported'
    params = {'symbol': symbol, 'token': finnhub_key}
    return requests.get(url=url, params=params)

In [4]:
response = get_fins_as_reported('AAPL')
r_json = response.json()

In [7]:
# r_json['data'][0]['report']['ic']

for i in r_json['data'][0]['report']['ic']:
    print(i['label'])

Net income
Change in foreign currency translation, net of tax
Change in fair value of derivatives
Adjustment for net (gains)/losses realized and included in net income
Total change in unrealized gains/losses on derivative instruments
Change in fair value of marketable securities
Adjustment for net (gains)/losses realized and included in net income
Total change in unrealized gains/losses on marketable securities
Total other comprehensive income/(loss)
Other comprehensive income/(loss):
Net sales
Total cost of sales
Gross margin
Research and development expense
Selling, general and administrative
Operating expenses:
Operating income
Other income/(expense), net
Total amounts reclassified from AOCI
Provision for income taxes
Basic (in dollars per share)
Diluted (in dollars per share)
Basic (in shares)
Diluted (in shares)


## Testing Polygon financials endpoint

In [2]:
fins = api_utils.get_financials('AAPL')
fins = fins['results']

In [14]:
# for i in range(len(fins)):
for key in fins[i].keys():
    print(key)
    # if key == 'reportPeriod':
    #     print(f'reportPeriod: {fins[i][key]}')

# print(fins[-1]['reportPeriod'])

reportPeriod: 2019-09-28
reportPeriod: 2018-09-29
reportPeriod: 2017-09-30
reportPeriod: 2016-09-24
reportPeriod: 2015-09-26
reportPeriod: 2014-09-27
reportPeriod: 2013-09-28
reportPeriod: 2012-09-29
reportPeriod: 2011-09-24
reportPeriod: 2010-09-25
